In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass('Enter your OpenAI API key: ')
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your Langchain API key: ")

os.environ["LANGCHAIN_TRACING_V2"]= "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="RAG-test"

from langchain_openai import ChatOpenAI, OpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=os.environ.get("OPENAI_API_KEY"))

In [44]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
import textwrap


In [79]:
import os

# Specify the absolute path to the target directory
target_directory_path = r"\Users\mava08\OneDrive - AF Gruppen ASA\Datamateriale-endringer\CCB"

# Get the current working directory
current_directory = os.getcwd()

# List all files in the target directory
files = os.listdir(target_directory_path)

for file_name in files:
    # Construct the absolute path to the file
    file_path = os.path.join(target_directory_path, file_name)
    
    # Check if it's a file and not a directory
    if os.path.isfile(file_path):
        # Find the relative path from the current directory to the file
        relative_path = os.path.relpath(file_path, start=current_directory)
        loader = PyPDFLoader(relative_path)
        pages = loader.load_and_split()

        
        # Split the text of the document
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(pages)
        
        # Add the splits to the vectorstore
        vectorstore = vectorstore.from_documents(documents=splits, embedding=OpenAIEmbeddings())
        
        

In [71]:
endring_systemvegg = """Beskrivelse av forholdet
Det er ikke medtatt systemvegger på høyde 3,4m.

Utredning og forslag til løsning
Det må bestilles plater på spesiallengde 3,4m.

Post 1: Minimumsantallet er 504 plater/ 1542m2. Plater på lengde 3,4m har en høyere pris med kr 9,30 pr m2.

Post 2: Det er rimeligere å benytte 3,4m platene til 3,3m enn å brenne inne med mye plater ekstra. Merkostnaden for nedkappet 3,3m plater er pr stk Kr 30,-. Det går med totalt 252plater på lengde 3,3m. F2 - 218 plater / D2 - 34 plater.

Post 3: Plater 3,4m som PT ikke benyttes i prosjektet blir 174stk.

Vederlagsjustering etter
 Justerte priser

 Eget pristilbud

Kostnadsoversikt
Postnr.	Beskrivelse	Firma	Enhet	Mengde	Enhetspris	Påslag	Totalbeløp	
1	Merkostnad pr m2 for spesiallengde plater	OBI	m2	1542	10,7	-	16 491,69	
2	Merkostnad nedkapping av 3,4 til 3,3	OBI	stk	252	34,5	-	8 694	
3	Merkostnad for 3,4m plater som PT ikke benyttes i prosjektet	OBI	stk	174	47,15	-	8 204,1	
Totalsum alle kostnader	33 389,79
"""


endring_himling = """
El.kanal gjennom himling

Beskrivelse av forholdet
Det skal monteres el-kanal gjennom himlingene

Utredning og forslag til løsning
Kappet vegglister for el-kanalene i bygg F 5-6-7-8 etg

Vederlagsjustering etter
 Regningsarbeid

Kostnadsoversikt
Postnr.	Beskrivelse	Firma	Enhet	Mengde	Enhetspris	Påslag	Totalbeløp	
1	arbeid	OBI	timer	8	660	-	5 280	

"""

endring_trespeilhimlinger = """Bygg DEG - trespilehimlinger i kommunikasjon

Beskrivelse av forholdet
Endret utførelse fra kontrakt post 10 EH.005

Mengder revidert da noe var uteglemt. endret fra 788m2 til 805,9m2

Utredning og forslag til løsning
Akustisk himlingsplate over trespilehimlinger

hvitpigmentering trespiler

Vederlagsjustering etter
 Kontraktens priser

 Eget pristilbud

Kostnadsoversikt
Postnr.	Beskrivelse	Firma	Enhet	Mengde	Enhetspris	Påslag	Totalbeløp	
1	Akustisk himlingsplate	OBI	m2	805.9	200	-	161 180	
2	hvitpigmentering trespiler	OBI	m2	805.9	41	-	33 041,9	
3	Prisregulering akustisk himlingsplate 1,3%	OBI	%	0.013	161 180	-	2 095,34	
4	R&D	OBI	%	0.05	196 316,9	-	9 815,85	
Totalsum alle kostnader	206 133,09"""


In [78]:
# Retrieve and generate using the relevant snippets of the blog.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

retriever = vectorstore.as_retriever()
mulitquery_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Prompt
template = """Du er en juridisk assistent som skal granske dokumenter for å sjekke om den innkommende endringsmeldingen
fra et selskap med krav om betaling er gyldig i henhold til relevante kontraktsdokumenter. De mener at de har utført arbeid som ikke er inkludert i kontrakten for byggeprosjektet. 
Dette betyr i all hovedsak at du skal sjekke at dette faktisk er en endring fra det som står beskrevet i kontrakten, og at dette medfører en ekstra kostnad. 
Har de rett på ekstra betaling, eller er arbeidet de ber om allerede inkludert i kontrakten, Begrunn svaret ditt og henvis til punkter i kontrakten og gi sitater.
hold svaret kort og konsist, og vær så presis som mulig.
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)



rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


response = rag_chain.invoke(endring_himling)

print(textwrap.fill(response, width=80))

Basert på kontraktdokumentene, spesifikt punkt 3.18 om hulltaking i himling, er
det klart at prisen for hulltaking og tilpasninger i himlinger er inkludert i
kontrakten. Derfor, hvis el-kanal gjennom himlingene innebærer hulltaking og
tilpasninger i himlinger, er dette allerede inkludert i kontrakten og selskapet
har ikke rett på ekstra betaling.
